In [1]:
repo_dir = "Repos"   # Set this to be where your github repos are located.
%load_ext autoreload
%autoreload 2

# Update the load path so python can find modules for the model
import sys
from pathlib import Path
sys.path.insert(0, str(Path.home() / repo_dir / "eye-ai-ml"))

In [2]:
# Prerequisites
import json
import os
from eye_ai.eye_ai import EyeAI

import pandas as pd
from pathlib import Path, PurePath
import logging

from deriva_ml import DatasetBag, Workflow, ExecutionConfiguration, DatasetVersion
from deriva_ml import MLVocab as vc
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', force=True)

2025-04-02 20:27:10.118959: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-02 20:27:10.119006: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-02 20:27:10.119798: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-02 20:27:10.126180: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-02 20:27:10.922480: W tensorflow/compiler/tf2

In [3]:
from deriva.core.utils.globus_auth_utils import GlobusNativeLogin
catalog_id = "eye-ai" #@param
host = 'www.eye-ai.org'


gnl = GlobusNativeLogin(host=host)
if gnl.is_logged_in([host]):
    print("You are already logged in.")
else:
    gnl.login([host], no_local_server=True, no_browser=True, refresh_tokens=True, update_bdbag_keychain=True)
    print("Login Successful")

You are already logged in.


In [4]:
cache_dir = '/data'
working_dir = '/data'
EA = EyeAI(hostname = host, catalog_id = catalog_id, cache_dir= cache_dir, working_dir=working_dir)

2025-04-02 20:27:11,961 - WARNING - nbstripout is not installed in repository. Please run nbstripout --install


In [5]:
# RID of source dataset, if any.
datasets = [
            '4-N9XE',
            '4-NAPT',
            '4-NBG6',
            '4-NC9J',
            '4-ND2Y', # 200
            '4-NDWY',
            '4-NFVT',
            '4-NHTP',
            '4-NKSJ',
            '4-NNRE', # 500
            '4-NQQY',
            '4-NVNA',
            '4-NZJP', # 1000
            '2-277G',
           ]

to_be_download = []
for dataset in datasets:
    ds_dict = {
        'rid': dataset,
        'materialize':True,
        'version':EA.dataset_version(dataset_rid=dataset),
    }
    to_be_download.append(ds_dict)


# EA.add_term(vc.workflow_type, "Create Dataset Workflow", description="A workflow to test creating a new dataset in eyeAI")
# Workflow instance
workflow_instance = EA.create_workflow(
    name="Dataset splitter creation",
    workflow_type="Create Dataset Workflow"
)
# Configuration instance.

# Set to False if you only need the metadata from the bag, and not the assets.
download_assets = False

config = ExecutionConfiguration(
    # Comment out the following line if you don't need the assets.
    datasets=to_be_download,
    workflow=workflow_instance,
    description="Splitting the original dataset.")

# Initialize execution
execution = EA.create_execution(config)

2025-04-02 20:27:13,436 - WARNING - File /home/nguyent8/Repos/eye-ai-exec/notebooks/Huy/DATA_SPLIT.ipynb has been modified since last commit. Consider commiting before executing
2025-04-02 20:27:14,568 - INFO - Materialize bag 4-N9XE... 
2025-04-02 20:27:15,624 - WARNING - nbstripout is not installed in repository. Please run nbstripout --install
2025-04-02 20:27:16,537 - INFO - Loading /data/4-N9XE_cf50778df0113494b8a4fcb9014793ae95741f9030231c76014081c91ad70a1d/Dataset_4-N9XE
2025-04-02 20:27:17,266 - INFO - Creating new database for dataset: 4-N9XE in /data/nguyent8/EyeAI_working/4-N9XE@32Z-ERBX-2HTG.db
2025-04-02 20:27:17,267 - INFO - Materialize bag 4-NAPT... 
2025-04-02 20:27:18,250 - WARNING - nbstripout is not installed in repository. Please run nbstripout --install
2025-04-02 20:27:19,380 - INFO - Loading /data/4-NAPT_a49c6bbc7bf0da2f95e450f2c43a9f1bd6cb1958023c3d2c723f01f16d90069d/Dataset_4-NAPT
2025-04-02 20:27:20,117 - INFO - Creating new database for dataset: 4-NAPT in /da

In [9]:
print(execution)

caching_dir: /data
_working_dir: /data/nguyent8/EyeAI_working
execution_rid: 4-S8JR
workflow_rid: 4-S8JM
asset_paths: []
configuration: datasets=[DatasetSpec(rid='4-N9XE', materialize=True, version=DatasetVersion(major=0, minor=3, patch=0)), DatasetSpec(rid='4-NAPT', materialize=True, version=DatasetVersion(major=0, minor=3, patch=0)), DatasetSpec(rid='4-NBG6', materialize=True, version=DatasetVersion(major=0, minor=3, patch=0)), DatasetSpec(rid='4-NC9J', materialize=True, version=DatasetVersion(major=0, minor=3, patch=0)), DatasetSpec(rid='4-ND2Y', materialize=True, version=DatasetVersion(major=0, minor=3, patch=0)), DatasetSpec(rid='4-NDWY', materialize=True, version=DatasetVersion(major=0, minor=3, patch=0)), DatasetSpec(rid='4-NFVT', materialize=True, version=DatasetVersion(major=0, minor=3, patch=0)), DatasetSpec(rid='4-NHTP', materialize=True, version=DatasetVersion(major=0, minor=3, patch=0)), DatasetSpec(rid='4-NKSJ', materialize=True, version=DatasetVersion(major=0, minor=3, p

In [29]:
ds_bag_list = [execution.datasets[i] for i in range(13)]
ds_bag_train = execution.datasets[13]

In [30]:
angle_2_df =  EA.filter_angle_2(ds_bag_train)
angle_2_df

,RID,RCT,RMT,RCB,RMB,URL,Filename,Description,Length,MD5,Observation,Image_ID,Source_Image,Image_Side,Image_Angle,Image_Tag
4,2-CXBA,2024-09-20 21:50:42.034449+00,2024-10-29 05:07:35.9238+00,https://auth.globus.org/b2541312-d274-11e5-913...,https://auth.globus.org/3769492a-b197-4063-952...,/hatrac/images/scans/subject/1001931/observati...,/data/2-277G_e2d15ade87945f60a5b9734a1a9117407...,,1795012,2d506c5624dde8ef386cf614f439330c,2-CN9G,15456028,,Right,2,Source
5,2-CXBC,2024-09-20 21:50:42.034449+00,2024-10-29 05:07:35.9238+00,https://auth.globus.org/b2541312-d274-11e5-913...,https://auth.globus.org/3769492a-b197-4063-952...,/hatrac/images/scans/subject/1001931/observati...,/data/2-277G_e2d15ade87945f60a5b9734a1a9117407...,,1795876,5c362b2e45f2e2fda79f58c4ea282685,2-CN9G,15456029,,Left,2,Source
6,2-D5KR,2024-09-20 21:50:47.214858+00,2024-10-29 05:07:39.143927+00,https://auth.globus.org/b2541312-d274-11e5-913...,https://auth.globus.org/3769492a-b197-4063-952...,/hatrac/images/scans/subject/382270/observatio...,/data/2-277G_e2d15ade87945f60a5b9734a1a9117407...,,454233,304d25700fc45e286416b352404225f3,2-CPNE,14910360,,Right,2,Source
11,2-D5M2,2024-09-20 21:50:47.214858+00,2024-10-29 05:07:39.143927+00,https://auth.globus.org/b2541312-d274-11e5-913...,https://auth.globus.org/3769492a-b197-4063-952...,/hatrac/images/scans/subject/382270/observatio...,/data/2-277G_e2d15ade87945f60a5b9734a1a9117407...,,472852,f859b4ac190bdbcfdfd9246737bf534c,2-CPNE,14910367,,Left,2,Source
12,2-D5PR,2024-09-20 21:50:47.214858+00,2024-10-29 05:07:39.143927+00,https://auth.globus.org/b2541312-d274-11e5-913...,https://auth.globus.org/3769492a-b197-4063-952...,/hatrac/images/scans/subject/419992/observatio...,/data/2-277G_e2d15ade87945f60a5b9734a1a9117407...,,1055578,b7fadc0c4428e63980fbd46da468f1b8,2-CPNY,15202735,,Right,2,Source
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28533,AHCJ,2023-05-03 11:26:38.224766+00,2024-10-29 05:08:08.780949+00,https://auth.globus.org/b2541312-d274-11e5-913...,https://auth.globus.org/3769492a-b197-4063-952...,/hatrac/images/scans/subject/999360/observatio...,/data/2-277G_e2d15ade87945f60a5b9734a1a9117407...,,742367,dc1e0d6212b5a020989e6fe6235ca09f,7M24,14349716,,Left,2,Source
28535,AHDE,2023-05-03 11:26:49.977089+00,2024-10-29 05:08:08.780949+00,https://auth.globus.org/b2541312-d274-11e5-913...,https://auth.globus.org/3769492a-b197-4063-952...,/hatrac/images/scans/subject/999623/observatio...,/data/2-277G_e2d15ade87945f60a5b9734a1a9117407...,,629479,6f4b305a9eaedd3fa4b81030a1c276d5,7HF0,14352451,,Right,2,Source
28538,AHDM,2023-05-03 11:26:53.009891+00,2024-10-29 05:08:08.780949+00,https://auth.globus.org/b2541312-d274-11e5-913...,https://auth.globus.org/3769492a-b197-4063-952...,/hatrac/images/scans/subject/999623/observatio...,/data/2-277G_e2d15ade87945f60a5b9734a1a9117407...,,696334,8922bd066d7146682c8f49c4dee072e5,7HF0,14352454,,Left,2,Source
28541,AHE6,2023-05-03 11:27:01.811931+00,2024-10-29 05:08:08.780949+00,https://auth.globus.org/b2541312-d274-11e5-913...,https://auth.globus.org/3769492a-b197-4063-952...,/hatrac/images/scans/subject/999943/observatio...,/data/2-277G_e2d15ade87945f60a5b9734a1a9117407...,,673876,1b13d6275cd30315bd3d5d3f9b686e7b,7KNT,14356088,,Left,2,Source


In [31]:
image_diag = ds_bag_train.get_table_as_dataframe('Image_Diagnosis')
image_diag = image_diag[image_diag['Diagnosis_Tag'] == 'Initial Diagnosis']
image_diag

,RID,RCT,RMT,RCB,RMB,Execution,Image,Feature_Name,Diagnosis_Image,Image_Quality,Diagnosis_Tag,Diagnosis_Status,Cup_Disk_Ratio,Comments,Process
0,4-6BNJ,2024-11-06 17:55:49.201939+00,2025-01-23 04:00:11.957746+00,https://auth.globus.org/2a32b19e-2728-4708-827...,https://auth.globus.org/3769492a-b197-4063-952...,4-6BNE,7VYG,Diagnosis,No Glaucoma,,Initial Diagnosis,,,,
1,4-6BNM,2024-11-06 17:55:49.201939+00,2025-01-23 04:00:11.957746+00,https://auth.globus.org/2a32b19e-2728-4708-827...,https://auth.globus.org/3769492a-b197-4063-952...,4-6BNE,7VYJ,Diagnosis,No Glaucoma,Good,Initial Diagnosis,,,,
2,4-6BNP,2024-11-06 17:55:49.201939+00,2025-01-23 04:00:11.957746+00,https://auth.globus.org/2a32b19e-2728-4708-827...,https://auth.globus.org/3769492a-b197-4063-952...,4-6BNE,7VYM,Diagnosis,No Glaucoma,,Initial Diagnosis,,,,
3,4-6BNR,2024-11-06 17:55:49.201939+00,2025-01-23 04:00:11.957746+00,https://auth.globus.org/2a32b19e-2728-4708-827...,https://auth.globus.org/3769492a-b197-4063-952...,4-6BNE,7VYP,Diagnosis,No Glaucoma,,Initial Diagnosis,,,,
4,4-6BNT,2024-11-06 17:55:49.201939+00,2025-01-23 04:00:11.957746+00,https://auth.globus.org/2a32b19e-2728-4708-827...,https://auth.globus.org/3769492a-b197-4063-952...,4-6BNE,7VYR,Diagnosis,No Glaucoma,Good,Initial Diagnosis,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28541,4-98EW,2024-11-06 17:55:49.201939+00,2025-01-23 04:00:39.670784+00,https://auth.globus.org/2a32b19e-2728-4708-827...,https://auth.globus.org/3769492a-b197-4063-952...,4-6BNE,7VYA,Diagnosis,No Glaucoma,Good,Initial Diagnosis,,,,
28542,4-98EY,2024-11-06 17:55:49.201939+00,2025-01-23 04:00:39.670784+00,https://auth.globus.org/2a32b19e-2728-4708-827...,https://auth.globus.org/3769492a-b197-4063-952...,4-6BNE,7VYC,Diagnosis,No Glaucoma,Good,Initial Diagnosis,,,,
28543,4-98F0,2024-11-06 17:55:49.201939+00,2025-01-23 04:00:39.670784+00,https://auth.globus.org/2a32b19e-2728-4708-827...,https://auth.globus.org/3769492a-b197-4063-952...,4-6BNE,7VYE,Diagnosis,No Glaucoma,,Initial Diagnosis,,,,
28544,4-98F2,2024-11-06 17:55:49.201939+00,2025-01-23 04:00:39.670784+00,https://auth.globus.org/2a32b19e-2728-4708-827...,https://auth.globus.org/3769492a-b197-4063-952...,4-6BNE,7Z7E,Diagnosis,No Glaucoma,,Initial Diagnosis,,,,


In [65]:
train_excluded_df = pd.read_csv("~/train_no_optic_disc_image_ids.csv")
train_excluded = train_excluded_df["ID"].tolist()

In [66]:
merged_df = pd.merge(angle_2_df, image_diag, left_on='RID', right_on='Image', how='inner')
df_filtered = merged_df[['Filename', 'Diagnosis_Image' ,'RID_x']]
df_filtered = df_filtered.rename(columns={'RID_x': 'RID'})
df_filtered = df_filtered[~df_filtered["RID"].isin(train_excluded)]
df_filtered

,Filename,Diagnosis_Image,RID
0,/data/2-277G_e2d15ade87945f60a5b9734a1a9117407...,Suspected Glaucoma,2-CXBA
1,/data/2-277G_e2d15ade87945f60a5b9734a1a9117407...,Suspected Glaucoma,2-CXBC
2,/data/2-277G_e2d15ade87945f60a5b9734a1a9117407...,Suspected Glaucoma,2-D5KR
3,/data/2-277G_e2d15ade87945f60a5b9734a1a9117407...,Suspected Glaucoma,2-D5M2
4,/data/2-277G_e2d15ade87945f60a5b9734a1a9117407...,Suspected Glaucoma,2-D5PR
...,...,...,...
9506,/data/2-277G_e2d15ade87945f60a5b9734a1a9117407...,Suspected Glaucoma,AHCJ
9507,/data/2-277G_e2d15ade87945f60a5b9734a1a9117407...,Suspected Glaucoma,AHDE
9508,/data/2-277G_e2d15ade87945f60a5b9734a1a9117407...,Suspected Glaucoma,AHDM
9509,/data/2-277G_e2d15ade87945f60a5b9734a1a9117407...,Suspected Glaucoma,AHE6


In [67]:
total_count = len(df_filtered['Diagnosis_Image'])
print(f"Total values in Diagnosis_Image: {total_count}")

# Count occurrences of each unique value
value_counts = df_filtered['Diagnosis_Image'].value_counts()
print("\nCounts of each unique value in Diagnosis_Image:")
print(value_counts)

diagnosis_values = df_filtered['Diagnosis_Image'].unique()
print("Values in Diagnosis_Image column:")
print(diagnosis_values)

Total values in Diagnosis_Image: 9442

Counts of each unique value in Diagnosis_Image:
Diagnosis_Image
No Glaucoma           5659
Suspected Glaucoma    3783
Name: count, dtype: int64
Values in Diagnosis_Image column:
['Suspected Glaucoma' 'No Glaucoma']


In [68]:
df_filtered

,Filename,Diagnosis_Image,RID
0,/data/2-277G_e2d15ade87945f60a5b9734a1a9117407...,Suspected Glaucoma,2-CXBA
1,/data/2-277G_e2d15ade87945f60a5b9734a1a9117407...,Suspected Glaucoma,2-CXBC
2,/data/2-277G_e2d15ade87945f60a5b9734a1a9117407...,Suspected Glaucoma,2-D5KR
3,/data/2-277G_e2d15ade87945f60a5b9734a1a9117407...,Suspected Glaucoma,2-D5M2
4,/data/2-277G_e2d15ade87945f60a5b9734a1a9117407...,Suspected Glaucoma,2-D5PR
...,...,...,...
9506,/data/2-277G_e2d15ade87945f60a5b9734a1a9117407...,Suspected Glaucoma,AHCJ
9507,/data/2-277G_e2d15ade87945f60a5b9734a1a9117407...,Suspected Glaucoma,AHDE
9508,/data/2-277G_e2d15ade87945f60a5b9734a1a9117407...,Suspected Glaucoma,AHDM
9509,/data/2-277G_e2d15ade87945f60a5b9734a1a9117407...,Suspected Glaucoma,AHE6


In [69]:
rids_in_other_ds = {200: [], 500: [], 1000: []}

for ds_bag_item in ds_bag_list:
    images = ds_bag_item.get_table_as_dataframe("Image")["RID"].tolist()
    key = len(images) /2 
    rid_list = rids_in_other_ds.get(key, [])
    rid_list.extend(images)
    rids_in_other_ds[key] = rid_list

In [94]:
import random, copy


def range_split(images):
    num_of_split = 0
    if images < 100:
        num_of_split = 10
    elif images >= 100 and images < 1000:
        num_of_split = 3
    elif images >= 1000 and images < 3000:
        num_of_split = 2
    else:
        num_of_split = 1
    return num_of_split

def split_dataset(df):
    # num_image_split = [10, 200, 500, 1000, 2000, 3000]
    num_image_split = [200, 500, 1000]
    res = {}

    suspected_glaucoma = df[df['Diagnosis_Image'] == 'Suspected Glaucoma']['RID'].tolist()
    no_glaucoma = df[df['Diagnosis_Image'] == 'No Glaucoma']['RID'].tolist()

    suspected_glaucoma = [rid for rid in suspected_glaucoma if rid not in train_excluded]
    no_glaucoma = [rid for rid in no_glaucoma if rid not in train_excluded]
        
    
    for num_images in num_image_split:
        current_subset_sets = []
        num_split = range_split(num_images)
        
        random.shuffle(suspected_glaucoma)
        random.shuffle(no_glaucoma)

          
        curr_suspected_glaucoma_rids = suspected_glaucoma
        curr_no_glaucoma_rids = no_glaucoma

        curr_suspected_glaucoma_rids = [rid for rid in curr_suspected_glaucoma_rids if rid not in rids_in_other_ds[num_images]]
        curr_no_glaucoma_rids = [rid for rid in curr_no_glaucoma_rids if rid not in rids_in_other_ds[num_images]]
        
        print(len(curr_suspected_glaucoma_rids))
        print(len(curr_no_glaucoma_rids))
        print("wat")
        for _ in range(num_split):
            if len(curr_suspected_glaucoma_rids) < num_images or len(curr_no_glaucoma_rids) < num_images:
                curr_suspected_glaucoma_rids = suspected_glaucoma
                curr_no_glaucoma_rids = no_glaucoma
                random.shuffle(curr_suspected_glaucoma_rids)
                random.shuffle(curr_no_glaucoma_rids)
            subset_suspected_glaucoma = curr_suspected_glaucoma_rids[:num_images]
            subset_no_glaucoma =  curr_no_glaucoma_rids[:num_images]
            concat = subset_suspected_glaucoma+subset_no_glaucoma
            current_subset_sets.append(concat)
            curr_suspected_glaucoma_rids =  curr_suspected_glaucoma_rids[num_images:]
            curr_no_glaucoma_rids =  curr_no_glaucoma_rids[num_images:]
                
        res[num_images] = current_subset_sets
    return res 

sets = split_dataset(df_filtered)

2783
4659
wat
1283
3159
wat
783
2659
wat


In [95]:
for key, value in sets.items():
    flattened = [item for sublist in value for item in sublist]
    all_unique = len(flattened) == len(set(flattened))
    print("Unique?", all_unique)
    print(key)
    print(len(value))
    print("Length dataset")
    for v in value:
        print(len(v))

Unique? True
200
3
Length dataset
400
400
400
Unique? False
500
3
Length dataset
1000
1000
1000
Unique? True
1000
2
Length dataset
2000
2000


In [96]:
data = {
    200: [],
    500: [],
    1000: [],
}

for ds_bag in ds_bag_list:
    df_image_len = int(len(ds_bag.get_table_as_dataframe('Image')['RID'].tolist())/2)
    if df_image_len in data:
        dataset_list = data[df_image_len]
    dataset_list.append(ds_bag)
    data[df_image_len] = dataset_list

In [97]:
ds_bag_master = {200 : '4-N9X6', 500: '4-NDWP', 1000: '4-NQQP'}

In [ ]:
with execution.execute() as exec:
    for key, value in sets.items():
        master_dataset = ds_bag_master[key]
        training_sets = []
        val = 6 if key == 200 or key == 500 else 4
        for i, item in enumerate(value, start=val):
            training_dataset = execution.create_dataset(['LAC', 'Training'], description=f'A training dataset of {key} images for each diagnosis, No {i}')
            EA.add_dataset_members(dataset_rid=training_dataset, members=item)
            training_sets.append(training_dataset)
        EA.add_dataset_members(dataset_rid=master_dataset, members=training_sets)

2025-04-03 00:30:08,874 - INFO - Start execution  ...


In [ ]:
execution.upload_execution_outputs(clean_folder=True)